In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=3053118b12d72162cb8b4713774f68036c65034f1e46d437f29dd82e09a708c8
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
# Import necessary libraries
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

# Load the dataset
dataset_name = 'ai4bharat/naamapadam'
raw_datasets = load_dataset(dataset_name, 'hi')


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
text_column_name = "tokens"
label_column_name = "ner_tags"
column_names = raw_datasets["train"].column_names
# print(column_names)
features = raw_datasets["train"].features
# print(features)

In [4]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [5]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    # print(examples.keys())  # Add this line to print the keys
    # print(examples.column_names)  # Add this line to print the column names
    tokenized_inputs = tokenizer(
        examples["tokens"],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
                # label_ids.append(label[min(word_idx, len(label) - 1)])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [6]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels )

2024-03-12 09:45:32.385763: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 09:45:32.385896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 09:45:32.504025: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
train_datasett = raw_datasets["train"]
train_sampled_variable=train_datasett.select(range(len(train_datasett)//49))
# train_sampled_variable = train_sampled_variable.map(
#     tokenize_and_align_labels,
#     batched=True,
#     num_proc=4,
#     load_from_cache_file=True,
#     desc="Running tokenizer on train dataset",
# )

test_size = int(len(raw_datasets['test']))
validation_size = int(len(raw_datasets['validation']))

test_sampled_variable = raw_datasets['test'].select(range(test_size))
validation_sampled_variable = raw_datasets['validation'].select(range(validation_size))

# test_sampled_variable = test_sampled
# validation_sampled_variable = validation_sampled

In [8]:
# train_dataset = train_sampled_variable
train_sampled_variable = train_sampled_variable.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/6 [00:00<?, ?ba/s]

In [9]:
# eval_dataset = validation_sampled_variable
validation_sampled_variable = validation_sampled_variable.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [10]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [11]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value

    return final_results

In [12]:
from transformers import TrainingArguments
training_args=TrainingArguments(
    output_dir='output_dir_bert',
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    learning_rate=2e-5
)

In [13]:
trainer = Trainer(
    model=model,
    train_dataset=train_sampled_variable,
    eval_dataset=validation_sampled_variable,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [14]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_la

In [15]:
try:
    train_result = trainer.train()
    metrics = train_result.metrics
except IndexError as e:
    print(f"IndexError occurred: {e}")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


**TOKENIZE MODULE**

In [16]:
# Tokenize the test dataset for prediction
def tokenize_dataset(dataset):
    tokenized_dataset = dataset.map(
        tokenize_and_align_labels,
        batched=True,
        num_proc=4,
        load_from_cache_file=True,
        desc="Running tokenizer on Test dataset",
    )
    return tokenized_dataset

**METRIC CALCULATOR**

In [17]:
# metrics_calculation.py

def calculate_macro_f1(metrics):
    # Extract precision, recall, and F1 score values for each label
    precision_LOC = metrics['test_LOC_precision']
    precision_ORG = metrics['test_ORG_precision']
    precision_PER = metrics['test_PER_precision']

    recall_LOC = metrics['test_LOC_recall']
    recall_ORG = metrics['test_ORG_recall']
    recall_PER = metrics['test_PER_recall']

    f1_LOC = metrics['test_LOC_f1']
    f1_ORG = metrics['test_ORG_f1']
    f1_PER = metrics['test_PER_f1']

    # Calculate macro precision, recall, and F1 score
    macro_precision = (precision_LOC + precision_ORG + precision_PER) / 3
    macro_recall = (recall_LOC + recall_ORG + recall_PER) / 3
    macro_f1 = (f1_LOC + f1_ORG + f1_PER) / 3

    # Compute the macro F1 score
    if macro_precision + macro_recall == 0:
        macro_f1_score = 0
    else:
        macro_f1_score = 2 * (macro_precision * macro_recall) / (macro_precision + macro_recall)

    return macro_f1_score


**PREDICTION + MACRO F1 SCORE CALCULATION ON TESTFILE**

In [18]:
# Code to tokenize test file for prediction
tokenized_test_set = tokenize_dataset(test_sampled_variable)

Running tokenizer on Test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
predictions, labels, metrics = trainer.predict(tokenized_test_set)

In [20]:
metrics

{'test_loss': 0.18198508024215698,
 'test_LOC_precision': 0.8003300330033003,
 'test_LOC_recall': 0.7899022801302932,
 'test_LOC_f1': 0.7950819672131147,
 'test_LOC_number': 614,
 'test_ORG_precision': 0.6421568627450981,
 'test_ORG_recall': 0.7485714285714286,
 'test_ORG_f1': 0.691292875989446,
 'test_ORG_number': 525,
 'test_PER_precision': 0.8333333333333334,
 'test_PER_recall': 0.8860759493670886,
 'test_PER_f1': 0.8588957055214723,
 'test_PER_number': 790,
 'test_overall_precision': 0.7667638483965015,
 'test_overall_recall': 0.8180404354587869,
 'test_overall_f1': 0.7915726109857035,
 'test_overall_accuracy': 0.9529985422007742,
 'test_runtime': 18.2327,
 'test_samples_per_second': 47.552,
 'test_steps_per_second': 4.004}

In [21]:
macro_f1 = calculate_macro_f1(metrics)
print(macro_f1)

0.7826106300457158


**MACRO F1 SCORE CALCULATION ON TRAINING DATA**

In [22]:
# Code to tokenize test file for prediction
tokenized_train_set = tokenize_dataset(train_sampled_variable)

Running tokenizer on Test dataset #3:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #0:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #2:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #1:   0%|          | 0/6 [00:00<?, ?ba/s]

In [23]:
predictions, labels, metrics = trainer.predict(tokenized_train_set)

In [24]:
macro_f1 = calculate_macro_f1(metrics)
print(macro_f1)

0.9120600681190091


**MACRO F1 SCORE CALCULATION FOR VALIDATION DATA**

In [25]:
# Code to tokenize test file for prediction
tokenized_validation_set = tokenize_dataset(validation_sampled_variable)

Running tokenizer on Test dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Test dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

In [26]:
predictions, labels, metrics = trainer.predict(tokenized_validation_set)

In [27]:
macro_f1 = calculate_macro_f1(metrics)
print(macro_f1)

0.7712329695476186


**TAKING PREDICTION ON 25 SENTENCES**

In [30]:
model.save_pretrained('my_indicNER')
tokenizer.save_pretrained("tokenizer_indicNER")

('tokenizer_indicNER/tokenizer_config.json',
 'tokenizer_indicNER/special_tokens_map.json',
 'tokenizer_indicNER/vocab.txt',
 'tokenizer_indicNER/added_tokens.json',
 'tokenizer_indicNER/tokenizer.json')

In [41]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [42]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

In [43]:
# Define the absolute path for the output file
output_file_path = "/kaggle/working/indicBERT_pred.txt"

#provide the finetuned model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/tokenizer_indicNER")
model = AutoModelForTokenClassification.from_pretrained("/kaggle/working/my_indicNER")

        
# Read the sentences from input.txt
with open("/kaggle/input/sentences-input/sentences.txt", "r", encoding="utf-8") as file:
    sentences = file.readlines()

# Open the output file for writing
with open(output_file_path, "w", encoding="utf-8") as output_file:
    # Iterate through each sentence
    for sentence in sentences:
        # Remove any leading or trailing whitespace and newline characters
        sentence = sentence.strip()
    
        # Predict labels for the current sentence
        predicted_labels = get_predictions(sentence=sentence, tokenizer=tokenizer, model=model)

        # Write the predicted labels to the output file, separated by spaces
        output_file.write(" ".join(predicted_labels) + "\n")

# Notify the user that the predictions have been saved
print("Predictions saved to indicBERT_pred.txt at", output_file_path)

Predictions saved to indicBERT_pred.txt at /kaggle/working/indicBERT_pred.txt


In [16]:
!rm -rf /kaggle/working/*